## Crater Lake Energy and Mass Balance (clemb)¶

(urban dictionary: Slang for a great big baddy)

### Motivation
* Develop a monitoring tool that estimates heat and steam input into crater lake routinely
* Learn from Tony about energy and mass balance calculation and provide a new home for his code
* Add interactivity to experiment with parameters
* Add noise model



In [1]:
import ipympl
import matplotlib.pyplot as plt
import os
from bokeh.plotting import output_notebook
from bokeh.io import push_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column, gridplot
from bokeh.models import DataRange1d, Range1d, LinearAxis,  ColumnDataSource, BoxZoomTool, ResetTool
import numpy as np
from scipy.stats import scoreatpercentile
import clemb
from clemb import LineInteractor
from ipyparallel import Client, Reference
import pandas as pd
import time
from IPython.display import display
import ipywidgets as widgets
output_notebook()

Loading BokehJS ...

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# Set the timeframe to analyse
start='1988-01-01'
end='1989-06-30'

In [4]:
c = Client()
dv = c[:]
dv.block = True
dv.clear()

In [5]:
with dv.sync_imports():
    import clemb

importing clemb on engine(s)


In [6]:
dv.execute("c = clemb.Clemb(clemb.LakeDataFITS(), clemb.WindDataCSV(),start='{}', end='{}')".format(start,end))

<AsyncResult: execute:finished>

In [7]:
def prep_data(dates, data, shape):
    _d = data.reshape(shape)
    df = {}
    df['date'] = dates
    left = dates
    right = left.copy()
    right[:-1] = left[1:]
    right[-1] = left[-1] + np.timedelta64(1,'D')
    df['left'] = left
    df['right'] = right
    df['median'] = np.median(_d,axis=0)
    df['min'], df['max'] = scoreatpercentile(_d,[16,84],axis=0)
    return ColumnDataSource(df)

In [8]:
def plot(res):
    tic = time.time()
    height = 200
    width = 800
    alpha = 0.5
    res1 = res.copy()
    res1[res1.fmelt<0.] = 0.0
    shape = (res.index.levels[0].size,res.index.levels[1].size)
    dates = np.array(res.index.levels[1],dtype=np.datetime64)
    t = prep_data(dates,res.loc[:,'t'].values,shape)
    w = prep_data(dates,res.loc[:,'wind'].values,shape)
    llvl = prep_data(dates,res.loc[:,'llvl'].values,shape)
    fmg = prep_data(dates,res.loc[:,'mg'].values,shape)
    fcl = prep_data(dates,res.loc[:,'cl'].values, shape)
    fm_valid = prep_data(dates,res1.loc[:,'fmelt'].values, shape)
    fm_invalid = prep_data(dates,res.loc[:,'fmelt'].values, shape)
    stm_valid = prep_data(dates,res1.loc[:,'pwr'].values, shape)
    stm_invalid = prep_data(dates,res.loc[:,'pwr'].values, shape) 
    
    tools = 'pan,box_zoom,resize,reset,save'

    # Mg++ and Cl- outflow
    p0 = figure(width=width, height=height, x_axis_type="datetime",toolbar_location='right', tools=tools,
                y_axis_label='Concentration [mg/l]')
    p0.extra_y_ranges = {"bar": Range1d(start=2400,end=2600)}
    p0.quad(left='left',right='right',bottom='min',top='max',color='navy',legend='Mg++',source=fmg,alpha=alpha,
           name='mgerr')
    p0.line(x='date',y='median',color='navy',source=fmg,name='mgm')
    p0.quad(left='left',right='right',bottom='min',top='max',color='green',legend='Cl-',source=fcl,alpha=alpha,
           name='clerr')
    p0.line(x='date',y='median',color='green',source=fcl,name='clm')
    p0.quad(left='left',right='right',bottom='min',top='max',color='black',legend='Lake level',source=llvl,alpha=alpha,
           name='llvlerr',y_range_name='bar')
    p0.line(x='date',y='median',color='black',source=llvl,y_range_name='bar',name='llvlm')
    p0.add_layout(LinearAxis(y_range_name="bar",axis_label='Lake level [m]',
                             major_label_text_color='black', major_tick_line_color='black'), 'right')

    
    # Temperature and wind
    p1 = figure(width=width, height=height, x_axis_type="datetime",toolbar_location='right', toolbar_sticky=False,
               y_axis_label='Temperature [°C]', x_range=p0.x_range, tools=tools)
    p1.extra_y_ranges = {"foo": Range1d(start=0, end=50)}
    p1.yaxis.major_tick_line_color='blue'
    p1.yaxis.major_label_text_color='blue'
    p1.quad(left='left',right='right',bottom='min',top='max',color='navy',source=t,alpha=alpha,name='terr',
           legend='Temp.')
    p1.line(x='date',y='median',color='navy',source=t,name='tm')
    p1.quad(left='left',right='right',bottom='min',top='max',color='green',y_range_name='foo',source=w, alpha=alpha,
           name='werr',legend='Wind')
    p1.line(x='date',y='median',color='green',source=w,y_range_name='foo',name='wm')
    p1.add_layout(LinearAxis(y_range_name="foo",axis_label='Wind speed [m/s]',
                             major_label_text_color='green', major_tick_line_color='green'), 'right')
    
    # Melt flow
    p2 = figure(width=width,height=height,x_axis_type="datetime",x_range=p0.x_range, toolbar_location='right',
               tools=tools)
    p2.quad(left='left',right='right',bottom='min',top='max',color='grey',source=fm_invalid,alpha=alpha,
           name='fmerr_iv')
    p2.line(x='date',y='median',color='grey',source=fm_invalid,name='fmm_iv')
    p2.quad(left='left',right='right',bottom='min',top='max',color='navy',source=fm_valid,alpha=alpha,
           name='fmerr_v')
    p2.line(x='date',y='median',color='navy',source=fm_valid,name='fmm_v')
    p2.yaxis.axis_label = 'Melt inflow [kT/day]'
    
    # Steam input
    p3 = figure(width=width, height=height, x_axis_type="datetime",x_range=p0.x_range, toolbar_location='right',
               tools=tools)
    p3.quad(left='left',right='right',bottom='min',top='max',source=stm_invalid,color='grey',alpha=alpha,
                   name='stmerr_iv')
    p3.line(x='date',y='median',color='grey',source=stm_invalid,name='stmm_iv')
    p3.quad(left='left',right='right',bottom='min',top='max',source=stm_valid,color='navy',alpha=alpha,
            name='stmerr_v')
    p3.line(x='date',y='median',color='navy',source=stm_valid,name='stmm_v')
    p3.yaxis.axis_label = 'Heat input [MW]'

   
    nh = show(gridplot([[p0,p2],[p1,p3]]), notebook_handle=True)
    toc = time.time()
    print("Plotting time: %.3f s"%(toc-tic))
    return (nh, p0,p1,p2,p3)

def update_plot(plots,res):
    tic = time.time()
    res1 = res.copy()
    res1[res1.fmelt<0.] = 0.0
    shape = (res.index.levels[0].size,res.index.levels[1].size)
    dates = np.array(res.index.levels[1],dtype=np.datetime64)
    t = prep_data(dates,res.loc[:,'t'].values,shape)
    w = prep_data(dates,res.loc[:,'wind'].values,shape)
    llvl = prep_data(dates,res.loc[:,'llvl'].values,shape)
    fmg = prep_data(dates,res.loc[:,'mg'].values,shape)
    fcl = prep_data(dates,res.loc[:,'cl'].values, shape)
    fm_valid = prep_data(dates,res1.loc[:,'fmelt'].values, shape)
    fm_invalid = prep_data(dates,res.loc[:,'fmelt'].values, shape)
    stm_valid = prep_data(dates,res1.loc[:,'pwr'].values, shape)
    stm_invalid = prep_data(dates,res.loc[:,'pwr'].values, shape)
    toc = time.time()
    #print("Data preparation took: {:.3f} s".format(toc-tic))
    tic = time.time()
    nh,p0,p1,p2,p3 = plots
    p0.select(name='mgerr')[0].data_source.data = fmg.data
    p0.select(name='mgm')[0].data_source.data = fmg.data
    p0.select(name='clerr')[0].data_source.data = fcl.data
    p0.select(name='clm')[0].data_source.data = fcl.data
    p0.select(name='llvlerr')[0].data_source.data = llvl.data
    p0.select(name='llvlm')[0].data_source.data = llvl.data

    p1.select(name='terr')[0].data_source.data = t.data
    p1.select(name='tm')[0].data_source.data = t.data
    p1.select(name='werr')[0].data_source.data = w.data
    p1.select(name='wm')[0].data_source.data = w.data

    p2.select(name='fmerr_iv')[0].data_source.data = fm_invalid.data
    p2.select(name='fmm_iv')[0].data_source.data = fm_invalid.data
    p2.select(name='fmerr_v')[0].data_source.data = fm_valid.data
    p2.select(name='fmm_v')[0].data_source.data = fm_valid.data
    
    p3.select(name='stmerr_iv')[0].data_source.data = stm_invalid.data
    p3.select(name='stmm_iv')[0].data_source.data = stm_invalid.data
    p3.select(name='stmerr_v')[0].data_source.data = stm_valid.data
    p3.select(name='stmm_v')[0].data_source.data = stm_valid.data  
  
    push_notebook(handle=nh)
    toc = time.time()
    #print("Updating the figure took: {:.3f} s".format(toc-tic))

In [9]:
def compute(dv,nsamples):
    tic = time.time()
    dv.scatter('ns', np.arange(nsamples),flatten=False)
    r = dv.apply(lambda ns: c.run(ns),Reference('ns'))
    rp = r[0]
    for i in range(1,len(dv)):
        if r[i] is None:
            continue
        rp = rp.append(r[i])
    toc = time.time()
    #print("Took %.3f s to calculate"%(toc-tic))
    return rp


In [10]:
rp = compute(dv,1)

In [11]:
xs = plot(rp)
nsamples = 1

def set_samples(sender):
    global nsamples
    nsamples = int(sender.value)

def compute_and_update(b):
    global rp
    dv.execute("c.update_data('{}','{}')".format(start,end))
    rp = compute(dv,nsamples)
    update_plot(xs,rp)
    
def update_temperature(change):
    val = change['new']
    dv.execute("t = c.get_variable('t')")
    if val < 0.1:
        dv.execute("t.std = None")
    else:
        dv.execute("t.std = {:.1f}".format(val))
        
def update_windspeed(change):
    val = change['new']
    dv.execute("w = c.get_variable('wind')")
    if val < 0.1:
        dv.execute("w.std = None")
    else:
        dv.execute("w.std = {:.1f}".format(val))
    
def update_enthalpy(change):
    val_min, val_max = change['new']
    # Note the default value for enthalpy has been set to 6.0
    val_min = 6.0 - val_min
    val_max = val_max - 6.0
    dv.execute("e = c.get_variable('enthalpy')")
    dv.execute("e.min = {:.1f}".format(val_min))
    dv.execute("e.max = {:.1f}".format(val_max))

def update_start(sender):
    global start
    try:
        start = str(np.datetime64(sender.value))
    except ValueError:
        print("Can't convert string '{}' into date.".format(sender.value))
    
    
def update_end(sender):
    global end
    try:
        end = str(np.datetime64(sender.value))
    except ValueError:
        print("Can't convert string '{}' into date.".format(sender.value))
 
def update_drcl(change):
    dv.execute("c.drcl = {}".format(change['new']))
    if dv['c.drmg'][0] == True:
        drcl.value = False
    
def update_drmg(change):
    dv.execute("c.drmg = {}".format(change['new']))
    if dv['c.drcl'][0] == True:
        drmg.value = False

w = widgets.Button(description='Compute', button_style='success', tooltip='Compute models and update the plot',layout=widgets.Layout(width='10%'))
expt = widgets.Button(description='Compute', button_style='success', tooltip='Compute models and update the plot')
t = widgets.Text(value='1',description='samples', layout=widgets.Layout(width='10%'))
ts = widgets.Text(value=start,description='start', layout=widgets.Layout(width='100%'))
te = widgets.Text(value=end,description='end', layout=widgets.Layout(width='100%'))
sl1 = widgets.FloatSlider(value=0.0, min=0, max=5.0, step=0.1, description='std(T)', orientation='horizontal',
                         continuous_update=False,layout=widgets.Layout(width='20%'))
sl2 = widgets.FloatSlider(value=0.0, min=0, max=10.0, step=0.1, description='std(WS)', orientation='horizontal',
                         continuous_update=False,layout=widgets.Layout(width='20%'))
sl3 = widgets.FloatRangeSlider(value=[6.0, 6.0], min=0, max=12, step=0.1, description='Enthalpy',
                               continuous_update=False, orientation='horizontal',layout=widgets.Layout(width='20%'))
drcl = widgets.Checkbox(value=False,description='Cl-',disabled=False)
drcl.observe(update_drcl,names='value')
drmg = widgets.Checkbox(value=False,description='Mg++',disabled=False)
drmg.observe(update_drmg,names='value')
t.on_submit(set_samples)
ts.on_submit(update_start)
te.on_submit(update_end)
w.on_click(compute_and_update)
sl1.observe(update_temperature,names='value')
sl2.observe(update_windspeed,names='value')
sl3.observe(update_enthalpy,names='value')
widgets.HBox([w,t,sl1,sl2,sl3,widgets.VBox([ts,te,widgets.HBox([drcl,drmg])])])

Plotting time: 0.190 s


In [12]:
import seaborn
from matplotlib.dates import date2num
import datetime

fig = plt.figure(figsize=(15,4))
dv.execute("d = c.get_variable('dv')")
d = dv['d'][0].data.copy()
drs = d.resample('2W').max()
x = date2num(drs.index.to_pydatetime())
y = drs.values
dates = np.array(rp.index.levels[1],dtype=np.datetime64)
shape = (rp.index.levels[0].size,rp.index.levels[1].size)
ll = prep_data(dates,rp.loc[:,'llvl'].values,shape)
x1 = date2num(dates.astype('M8[ms]').astype('O'))
y1 = ll.data['median']
ax = fig.add_subplot(211)
l = ax.plot_date(x,y,'k')
p = LineInteractor(ax, l[0])
ax.set_ylim(0.99,1.01)
ax.set_ylabel('Dilution factor')
ax1 = fig.add_subplot(212)
ax1.plot_date(x1,y1,'r')
ax1.set_ylabel('Lake level [m]')
plt.show()


def update_dilution(b):
    x,y = p.get_data()
    nd = pd.Series(y,index=drs.index)
    nd = nd.reindex(d.index,method='nearest')
    dv['tmp']=clemb.Gauss(nd)
    dv.execute("dd = c.get_variable('dv')")
    dv.execute('dd.data = tmp.data')
    
def reset_dilution(b):
    p.reset()

u = widgets.Button(description='Update', button_style='success', tooltip='Update the dilution factor')
r = widgets.Button(description='Reset', button_style='success', tooltip='Reset dilution factor')
u.on_click(update_dilution)
r.on_click(reset_dilution)
widgets.HBox([u,r])


## Workplan

### Next iteration
* retrieve wind data from MetService
* smooth input parameters
* allow setting input parameters to their default values

### 3rd iteration
* stand-alone browser interface
* run automatically in the background
* make heat flux values available to users
* allow taking snapshots of interactive session

### 4th iteration
* develop forward model

### 5th iteration
* make applicable to other volcanic lakes
* test different evaporation models
